In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/PANDAS_PIPELINE'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
# Read in our data
data = pd.read_csv('NFL Play by Play 2009-2018 (v5).csv')
data.head()

/home/chris/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [4]:
import re
from operator import itemgetter

In [5]:
def recognize_step_i(i, **kwargs):
    fun_string = "step_{}".format(i)
    arg_string = "arg_{}".format(i)
    fun_1 = itemgetter(fun_string)(kwargs)
    arg_dict = itemgetter(arg_string)(kwargs)
    return fun_1, arg_dict

In [6]:
def dictionary_apply_kwarg_funs(**kwargs):
    keys = " ".join(kwargs.keys())
    num_steps = len(re.findall("step_", keys))
    data_ = kwargs['input_']
    output_ = {}
    
    if data_ is not None:
        intermediate = data_
        for i in range(1, num_steps + 1):
            step_ID = "step_{}".format(i)            
            fun_, arg_dict_ = recognize_step_i(i, **kwargs)
            intermediate = fun_(intermediate,**arg_dict_)
            output_[step_ID] = intermediate
    else:
        print("Input data is empty")
    return output_, data

In [7]:
kwargs = {'input_':data,'step_1':pd.DataFrame.dropna, 'arg_1':{'axis':1,'thresh':1},
         'step_2':pd.DataFrame.dropna, 'arg_2':{'axis':1,'thresh':2},
    'step_3':pd.DataFrame.dropna, 'arg_3':{'axis':1,'thresh':3}}
output_,input_ = dictionary_apply_kwarg_funs(**kwargs)